<a href="https://colab.research.google.com/github/debajyoti22/Bart-Simcls-Paraphrase-Model/blob/main/BartSimclsParaphrase.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install nltk 
!pip install numpy
!pip install matplotlib
!pip install absl-py
!pip install sacrebleu

In [ ]:
! pip install datasets transformers[sentencepiece]
!pip install transformers datasets sentencepiece
! pip install nltk rouge_score

In [ ]:
!git clone https://github.com/andrejmiscic/simcls-pytorch.git
!cp -R simcls-pytorch/src src/

In [ ]:
!pip install src.model
from src.model import SimCLS, GeneratorType

In [ ]:
! pip install sentence-splitter
! pip install SentencePiece
!pip install transformers

In [ ]:
import torch
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
from transformers import BartTokenizer, BartForConditionalGeneration




model = BartForConditionalGeneration.from_pretrained('Yale-LILY/brio-cnndm-uncased')
tokenizer = BartTokenizer.from_pretrained('Yale-LILY/brio-cnndm-uncased')
ARTICLE_TO_SUMMARIZE = """
This week MME speaks to the man charged with boosting growth in the Kingdom of Bahrain: Sheikh Mohammed bin Isa Al Khalifa. Sheikh Mohammed is concerned the nuclear dispute with Iran is deterring potential investors . As chief executive of the Economic Development Board, one of Sheikh Mohammed's main tasks is to ensure Bahrain stays ahead of its competitors. The government's pushing ahead with economic reforms to reduce the Kingdom's dependence on oil. But with crude prices near $100 a barrel, John Defterios asked whether the appetite for change had diminished. SM: The reforms are being done for aspirational reasons rather than necessity, so we are trying to enter the post-oil area because we want to. And therefore, because of this aspirational aspect, trying to get to as opposed to people pushing us somewhere we don't want to go. JD: How do you redefine Bahrain's role? Traditionally it's been a financial center, but you have Qatar, Saudi Arabia, Dubai even Abu Dhabi emerging in financial services. How you redefine it to sustain a lead? SM: The rising tide lifts a lot of places and as the region is booming and growing, financial services are growing throughout the region. Bahrain's strengths have been and will continue to be its human capital, the people that have led the financial sector and the long standing regulations that have been established in Bahrain. JD: Strategically and a geopolitical question here: the Crown Prince came out and declared what Iran's intentions really are -- in his view -- for a nuclear policy, and called for greater diplomacy right ahead of a visit by the President of Iran to the country. What's the motivation here? SM: We feel that the distraction caused by the Iranian nuclear issue is holding investment back in the region as a whole, and increasing the security threat of the region. Nobody is debating the right to peaceful nuclear power, but we all have that need and we all need a peaceful resolution to this. JD: Historically, Iran has even made claims against Bahrain, what do you do now moving forward now? Do you bring Russia, China, the United States and the European Union closer together, in terms of where we should be going in the future? SM: We've resolved our historical issue with Iran, and we are well beyond that, but going forward... This is such a important issue affecting security worldwide, that we feel it is important and at a level high enough that we need all powers involved because it does affect everybody. JD: Can the GCC (Gulf Cooperation Council) become much tighter as an organization to have a voice in that process; as opposed to speaking from the sidelines and hoping that the major powers move forward? SM: Absolutely, I mean, the GCC is speaking with a unified voice. The Kingdom of Saudi Arabia came out with a proposal for providing nuclear fuel from an independent authority; this was completely backed from the GCC. We are all moving forward and are willing to play a beneficial role to resolve this issue. Meet the Sheikh: . Sheikh Mohammed bin Isa Al Khalifa is chief executive of the Economic Development Board, the organization responsible for stimulating and supporting growth of the Bahraini economy. He was appointed in June 2005, following 10 years working directly for His Highness Sheikh Salman bin Hamad Al Khalifa, Crown Prince, Commander-in-Chief of the Bahrain Defense Force and Chairman of the Economic Development Board. From 1995 to 1999, he was director of the Crown Prince's office. In 1999, he was appointed head of the court and also deputy head of the Crown Prince's International Scholarship Program. During this time, Sheikh Mohammed was heavily involved with the Crown Prince's initiatives to liberalize the telecommunications sector and with labor market and economic reforms. A keen motor sports enthusiast, Sheikh Mohammed was deputy board chairman of the Bahrain International Circuit and is a former vice president of the Bahrain Motor Club, now known as the Bahrain Motor Federation. Sheikh Mohammed, who is married with two sons and one daughter, was born on April 14, 1969. In 1992, he received a bachelor's degree in economic theory from the American University, Washington DC. Two years later, he received a post-graduate diploma in business studies from the London School of Economics.
"""
#baart



article = ARTICLE_TO_SUMMARIZE
inputs = tokenizer([article], max_new_tokens = 1024, return_tensors="pt", truncation=True)
# Generate Summary
summary_ids = model.generate(inputs["input_ids"])
summary = tokenizer.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
print(summary)

In [ ]:
summarizer = SimCLS(generator_type=GeneratorType.Bart,
                    generator_path="facebook/bart-large-cnn",
                    scorer_path="andrejmiscic/simcls-scorer-cnndm")

In [ ]:
summary2 = summarizer(article)
summary2

In [ ]:
model_name = 'tuner007/pegasus_paraphrase'
torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name).to(torch_device)


def get_response(input_text,num_return_sequences):
  batch = tokenizer.prepare_seq2seq_batch([input_text],truncation=True,padding='longest',max_length=60, return_tensors="pt").to(torch_device)
  translated = model.generate(**batch,max_length=60,num_beams=10, num_return_sequences=num_return_sequences, temperature=1.5)
  tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
  return tgt_text

In [ ]:
context = summary2

In [ ]:
from sentence_splitter import SentenceSplitter, split_text_into_sentences

splitter = SentenceSplitter(language='en')

sentence_list = splitter.split(context)
sentence_list

In [ ]:
paraphrase = []

for i in sentence_list:
  a = get_response(i,1)
  paraphrase.append(a)

In [ ]:
paraphrase

In [ ]:
paraphrase2 = [' '.join(x) for x in paraphrase]
paraphrase2

In [ ]:
paraphrase3 = [' '.join(x for x in paraphrase2) ]
paraphrased_text = str(paraphrase3).strip('[]').strip("'")
paraphrased_text

In [ ]:
print(paraphrased_text)